# Resumo do código

### <u>Código que gera os ficheiros para estudo do comportamento da Delta</u>
---
O objectivo é receber dados da Delta e devolver um conjunto de métricas para prever roturas. Devolve um ficheiro que pode entrar no código 1 para juntar aos dados dos ninjas.

---
- Inputs

> __Dados completos da Delta em pastas de ficheiros__ (de azul a verde)
> - Stocks e trânsito, Sellout do dia anterior

                    ou

> __Ficheiro já completo__ (de vermelho a verde)
> - Stocks e trânsito, Sellout do dia anterior

- Outputs

> __Ficheiro com produtos em causa__ em formato Long

> __Métricas novas:__
> - Roturas de Stock e Pré-rotura
> - Sinal
> - Ciclos e Adequação de Stock
> - MSA (média de sellouts 10 dias antes)
> - STK (Stock disponível + trânsito)
> - (Novo) Balanço médio, mediano, liberal e conservador 
> - (Novo) Dias para a rotura de stock e de prateleira


In [86]:
%%time
import pandas as pd
import numpy as np
import datetime


def escrever_csv(dfa, nome): 
    dfa.to_csv('D:\\B&N Dados\\Delta\\Piloto\\%s.csv' %nome, index=False)

CPU times: total: 0 ns
Wall time: 0 ns


In [89]:
oi=pd.read_csv('D:\\B&N Dados\\Delta\\Piloto\\Métricas3060120_Piloto.csv')

In [90]:
oi

,DATA,EAN,DESC_ARTIGO,STORE,STORE_NAME,INTRANSIT,EXPECTED,PRES_STOCK,STOCK,STOCK_1_Dias_Antes,...,Balance_Conservador_1_Dias_Antes,Balance_Conservador_2_Dias_Antes,Balance_Conservador_3_Dias_Antes,Balance_Conservador_4_Dias_Antes,Balance_Conservador_5_Dias_Antes,Balance_Conservador_6_Dias_Antes,Balance_Conservador_7_Dias_Antes,Balance_Conservador_8_Dias_Antes,Balance_Conservador_9_Dias_Antes,Balance_Conservador_10_Dias_Antes
0,2023-04-14,5609060007087,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,1,CNT MATOSINHOS,0,0,56,74.0,80.0,...,0.400000,0.545455,0.633663,0.761905,0.897196,0.770642,0.512000,0.517986,0.347222,0.320388
1,2023-04-15,5609060007087,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,1,CNT MATOSINHOS,0,0,56,59.0,74.0,...,0.432432,0.600000,0.727273,0.792079,0.914286,1.046729,0.880734,0.576000,0.575540,0.381944
2,2023-04-16,5609060007087,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,1,CNT MATOSINHOS,0,0,56,46.0,59.0,...,0.542373,0.648649,0.800000,0.909091,0.950495,1.066667,1.196262,0.990826,0.640000,0.633094
3,2023-04-21,5609060007087,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,1,CNT MATOSINHOS,0,84,56,143.0,65.0,...,0.461538,0.483871,1.185185,1.600000,2.086957,1.898305,1.729730,1.800000,1.818182,1.742574
4,2023-04-22,5609060007087,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,1,CNT MATOSINHOS,0,0,56,135.0,143.0,...,0.132159,0.692308,0.645161,1.481481,1.920000,2.434783,2.169492,1.945946,2.000000,2.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,2023-04-29,5609060007070,CEVADA SOLÚVEL DELTA FRASCO 200G,459,CNT BRAGA,0,0,64,582.0,610.0,...,0.055300,0.115016,0.222576,0.683646,0.796875,2.462069,2.472727,2.428571,2.405660,2.407725
1196,2023-04-30,5609060007070,CEVADA SOLÚVEL DELTA FRASCO 200G,459,CNT BRAGA,0,0,64,566.0,582.0,...,0.082474,0.082949,0.153355,0.278219,0.820375,0.929688,2.813793,2.781818,2.698413,2.646226
1197,2023-05-05,5609060007070,CEVADA SOLÚVEL DELTA FRASCO 200G,459,CNT BRAGA,0,276,64,243.0,221.0,...,0.547284,0.620438,0.266075,0.261194,0.296820,0.288660,0.221198,0.345048,0.556439,1.504021
1198,2023-05-06,5609060007070,CEVADA SOLÚVEL DELTA FRASCO 200G,459,CNT BRAGA,0,252,64,328.0,243.0,...,0.524085,0.820926,0.827251,0.332594,0.313433,0.346290,0.329897,0.248848,0.383387,0.612083


---

#  <span style="color:red"><u>Ler Ficheiro Completo</u> </span>

In [2]:
%%time

#Ler os ficheiros
df_2022 = pd.read_csv('D:\\B&N Dados\\Delta\\Stocks\\Stocks2022\\Stocks_Delta_2022_Limpo.csv')
df_2023 = pd.read_csv('D:\\B&N Dados\\Delta\\Stocks\\Stocks2023\\Stocks_Delta_2023_Limpo.csv')

# Juntar as bases
dataframes = [df_2022, df_2023]
df_Fusão = pd.concat(dataframes, ignore_index=True)
df_Fusão['DATA']= pd.to_datetime(df_Fusão['DATA'], format='%Y-%m-%d')

# Ficheiro de previsão
df_Prophet = pd.read_csv('D:\\B&N Dados\\Delta\\Forecast\\Prophet.csv')
df_Prophet['DATA']= pd.to_datetime(df_Prophet['DATA'], format='%Y-%m-%d')

df_XGBoost = pd.read_csv('D:\\B&N Dados\\Delta\\Forecast\\XGBoost.csv')
df_XGBoost['DATA']= pd.to_datetime(df_XGBoost['DATA'], format='%Y-%m-%d')

CPU times: total: 26.6 s
Wall time: 34.2 s


# <font color=red>Opcional:</font> Definir produtos em causa

- Produtos específicos

In [79]:
%%time
# Ler ficheiro dos produtos e lojas para dataframe
df_produtos = pd.read_csv('D:\\B&N Dados\\Delta\\Piloto\\produtos.txt', header=None)
df_lojas = pd.read_csv('D:\\B&N Dados\\Delta\\Piloto\\lojas.txt', header=None)

# Passar para uma lista
produtos = df_produtos[0].tolist()
lojas = df_lojas[0].tolist()

# Alterar o dataframe para apenas incluir os produtos e lojas em causa
dfFinal = df_Fusão[(df_Fusão["DESC_ARTIGO"].isin(produtos)) & (df_Fusão["STORE_NAME"].isin(lojas))].copy()

CPU times: total: 875 ms
Wall time: 869 ms


In [27]:
dfFinal['SELLOUT'] = dfFinal.groupby(["STORE","EAN"])['SELLOUT_1_Dias_Antes'].shift(-1)
dfFinal['STOCK'] = dfFinal.groupby(["STORE","EAN"])['STOCK_1_Dias_Antes'].shift(-1)

In [28]:
dfFinal.head()

,DATA,EAN,DESC_ARTIGO,STORE,STORE_NAME,INTRANSIT,EXPECTED,PRES_STOCK,STOCK,STOCK_1_Dias_Antes,SELLOUT,SELLOUT_1_Dias_Antes
60608,2022-01-01,5609060007087,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,1,CNT MATOSINHOS,0,48,120,151.0,151.0,0.0,11.0
60609,2022-01-02,5609060007087,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,1,CNT MATOSINHOS,0,48,120,137.0,151.0,14.0,0.0
60610,2022-01-03,5609060007087,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,1,CNT MATOSINHOS,48,0,120,175.0,137.0,10.0,14.0
60611,2022-01-04,5609060007087,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,1,CNT MATOSINHOS,0,0,120,162.0,175.0,13.0,10.0
60612,2022-01-05,5609060007087,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,1,CNT MATOSINHOS,0,48,120,195.0,162.0,15.0,13.0


## <font color=red>Fim</font> 

- Fundir o Ficheiro da Delta com a previsão

In [29]:
%%time
# Prophet
dfFinal = pd.merge(dfFinal, df_Prophet[['DATA', 'STORE', 'DESC_ARTIGO', 'Prophet']], how="left", on=['DATA', 'STORE', 'DESC_ARTIGO',] )
# XGBoost
dfFinal = pd.merge(dfFinal, df_XGBoost[['DATA', 'STORE', 'DESC_ARTIGO', 'XGBoost']], how="left", on=['DATA', 'STORE', 'DESC_ARTIGO',] )

CPU times: total: 562 ms
Wall time: 557 ms


# <font color=green>Ficheiro Lido<font>

---

# Colunas de métricas interessantes

> - ROTURA

In [30]:
# Definir coluna de rotura (se stock menor ou igual a 0 e existe Linear)

dfFinal["ROTURA"] = np.where((dfFinal["STOCK"] <= 0) & (dfFinal["PRES_STOCK"] > 0), 1, 0)

> - PRÉ_ROTURA

In [31]:
# Definir coluna de rotura (se stock menor ou igual a 0)

dfFinal["PRE_ROTURA"] = (dfFinal["STOCK"] < dfFinal["PRES_STOCK"]).astype(int)

# Colunas de métricas 30, 60, 120 e 180 dias antes

> - Volatilidade de Procura: <br>
coeficiente de variação

In [32]:
dfFinal["Volatilidade_30"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=30, min_periods=1).std())/
                             dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=30, min_periods=1).mean()))

dfFinal["Volatilidade_60"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=60, min_periods=1).std())/
                             dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=60, min_periods=1).mean()))

dfFinal["Volatilidade_120"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=120, min_periods=1).std())/
                              dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=120, min_periods=1).mean()))

dfFinal["Volatilidade_180"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=180, min_periods=1).std())/
                              dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=180, min_periods=1).mean()))



> - Percentagem de Rotura: <br>
média de roturas $* 100$

In [33]:
#historico("Percentagem_Roturas", "ROTURA", pd.Series.mean)

dfFinal["Percentagem_Roturas_30"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA'].shift(1).transform(lambda x: x.rolling(window=30, min_periods=1).mean()))*100

dfFinal["Percentagem_Roturas_60"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA'].shift(1).transform(lambda x: x.rolling(window=60, min_periods=1).mean()))*100

dfFinal["Percentagem_Roturas_120"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA'].shift(1).transform(lambda x: x.rolling(window=120, min_periods=1).mean()))*100

dfFinal["Percentagem_Roturas_180"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA'].shift(1).transform(lambda x: x.rolling(window=180, min_periods=1).mean()))*100


> - Percentagem de Supply:<br>
média de vezes que foi pedido stock $*100$

In [34]:
dfFinal["New_Supply"] = np.where((dfFinal["EXPECTED"].shift(1)==0) & (dfFinal["EXPECTED"]>0), 1, 0)

In [35]:
#historico("Percentagem_Supply", "New_Supply", pd.Series.mean)

dfFinal["Percentagem_Supply_30"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['New_Supply'].shift(1).transform(lambda x: x.rolling(window=30, min_periods=1).sum())/30)*100

dfFinal["Percentagem_Supply_60"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['New_Supply'].shift(1).transform(lambda x: x.rolling(window=60, min_periods=1).sum())/60)*100

dfFinal["Percentagem_Supply_120"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['New_Supply'].shift(1).transform(lambda x: x.rolling(window=120, min_periods=1).sum())/120)*100

dfFinal["Percentagem_Supply_180"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['New_Supply'].shift(1).transform(lambda x: x.rolling(window=180, min_periods=1).sum())/180)*100


> - Efeito fim de semana

In [36]:
dfFinal['SELLOUT_fds'] = dfFinal[dfFinal['DATA'].dt.weekday.isin([4,5,6])]["SELLOUT"].copy()
dfFinal['SELLOUT_semana'] = dfFinal[dfFinal['DATA'].dt.weekday.isin([0,1,2,3])]["SELLOUT"].copy()

In [37]:
dfFinal["Efeito_Fds_30"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds'].shift(1).transform(lambda x: x.rolling(window=30, min_periods=1).median())/
                            (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana'].shift(1).transform(lambda x: x.rolling(window=30, min_periods=1).median())))-1

dfFinal["Efeito_Fds_60"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds'].shift(1).transform(lambda x: x.rolling(window=60, min_periods=1).median())/
                            (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana'].shift(1).transform(lambda x: x.rolling(window=60, min_periods=1).median())))-1

dfFinal["Efeito_Fds_120"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds'].shift(1).transform(lambda x: x.rolling(window=120, min_periods=1).median())/
                            (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana'].shift(1).transform(lambda x: x.rolling(window=120, min_periods=1).median())))-1

dfFinal["Efeito_Fds_180"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds'].shift(1).transform(lambda x: x.rolling(window=180, min_periods=1).median())/
                            (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana'].shift(1).transform(lambda x: x.rolling(window=180, min_periods=1).median())))-1

> - Tempo médio inter-supply

In [38]:
dfFinal["InterSupply"] = np.where(dfFinal["EXPECTED"]==0, 1, 0)

groups = (dfFinal['InterSupply'] != dfFinal['InterSupply'].shift()).cumsum()
result = dfFinal.groupby(groups).agg({'DATA': 'first', 'DESC_ARTIGO': 'first', 'STORE': 'first', 'InterSupply': 'sum'}).reset_index(drop=True)
result = result[result['InterSupply'] > 0]

dfFinal = dfFinal.drop(columns=['InterSupply'])

dfFinal = pd.merge(dfFinal, result, how="left", on=["DATA","DESC_ARTIGO", "STORE"])

In [39]:
#historico("InterSupplyMed", "InterSupply", pd.Series.mean)

dfFinal["InterSupplyMed_30"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['InterSupply'].transform(lambda x: x.rolling(window=30, min_periods=1).mean()))*100

dfFinal["InterSupplyMed_60"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['InterSupply'].transform(lambda x: x.rolling(window=60, min_periods=1).mean()))*100

dfFinal["InterSupplyMed_120"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['InterSupply'].transform(lambda x: x.rolling(window=120, min_periods=1).mean()))*100

dfFinal["InterSupplyMed_180"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['InterSupply'].transform(lambda x: x.rolling(window=180, min_periods=1).mean()))*100


> - Tempo indisponível

In [41]:
'''O que vai acontecer é: no primeiro dia em que há rotura vai aparecer a soma de todos os dias com rotura a seguir a esse!
Todos os outros valores serão NaN para não serem considerados quando for feita a média. Assim a média corresponderá ao
número médio de dias em que se deixa um produto em rotura.'''

dfFinal["Tempo_Indisponível"] = np.where(dfFinal["ROTURA"]==1, 1, 0)

groups = (dfFinal['Tempo_Indisponível'] != dfFinal['Tempo_Indisponível'].shift()).cumsum()
result = dfFinal.groupby(groups).agg({'DATA': 'first', 'DESC_ARTIGO': 'first', 'STORE': 'first', 'Tempo_Indisponível': 'sum'}).reset_index(drop=True)
result = result[result['Tempo_Indisponível'] > 0]

dfFinal = dfFinal.drop(columns=['Tempo_Indisponível'])

dfFinal = pd.merge(dfFinal, result, how="left", on=["DATA","DESC_ARTIGO", "STORE"])

In [42]:
#historico("Percentagem_Supply", "New_Supply", pd.Series.mean)

dfFinal["Tempo_Indisponível_30"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Tempo_Indisponível'].shift(1).transform(lambda x: x.rolling(window=30, min_periods=1).mean()))

dfFinal["Tempo_Indisponível_60"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Tempo_Indisponível'].shift(1).transform(lambda x: x.rolling(window=60, min_periods=1).mean()))

dfFinal["Tempo_Indisponível_120"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Tempo_Indisponível'].shift(1).transform(lambda x: x.rolling(window=120, min_periods=1).mean()))

dfFinal["Tempo_Indisponível_180"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Tempo_Indisponível'].shift(1).transform(lambda x: x.rolling(window=180, min_periods=1).mean()))


> - Percentagem de dias em Stock Borderline

In [43]:
dfFinal["Stock_Borderline"] = np.where(dfFinal["STOCK"]<0.2*dfFinal["PRES_STOCK"], 1, 0)

In [44]:
dfFinal["Tempo_Stock_Borderline_30"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Stock_Borderline'].shift(1).transform(lambda x: x.rolling(window=30, min_periods=1).mean()))*100

dfFinal["Tempo_Stock_Borderline_60"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Stock_Borderline'].shift(1).transform(lambda x: x.rolling(window=60, min_periods=1).mean()))*100

dfFinal["Tempo_Stock_Borderline_120"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Stock_Borderline'].shift(1).transform(lambda x: x.rolling(window=120, min_periods=1).mean()))*100

dfFinal["Tempo_Stock_Borderline_180"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Stock_Borderline'].shift(1).transform(lambda x: x.rolling(window=180, min_periods=1).mean()))*100


> - Percentagem de dias de Linear Incompleto

In [45]:
dfFinal["Linear_Incompleto"] = np.where(dfFinal["STOCK"]<dfFinal["PRES_STOCK"], 1, 0)

In [46]:
dfFinal["Tempo_Linear_Incompleto_30"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Linear_Incompleto'].shift(1).transform(lambda x: x.rolling(window=30, min_periods=1).mean()))*100

dfFinal["Tempo_Linear_Incompleto_60"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Linear_Incompleto'].shift(1).transform(lambda x: x.rolling(window=60, min_periods=1).mean()))*100

dfFinal["Tempo_Linear_Incompleto_120"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Linear_Incompleto'].shift(1).transform(lambda x: x.rolling(window=120, min_periods=1).mean()))*100

dfFinal["Tempo_Linear_Incompleto_180"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Linear_Incompleto'].shift(1).transform(lambda x: x.rolling(window=180, min_periods=1).mean()))*100


> - Percentagem de dias sem vendas

In [50]:
dfFinal["Sem_Vendas"] = np.where(dfFinal["SELLOUT"] == 0, 1, 0)

In [51]:
#historico("Sem_Vendas", "Sem_Vendas", pd.Series.mean)

dfFinal["Sem_Vendas_30"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Sem_Vendas'].shift(1).transform(lambda x: x.rolling(window=30, min_periods=1).mean()))*100

dfFinal["Sem_Vendas_60"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Sem_Vendas'].shift(1).transform(lambda x: x.rolling(window=60, min_periods=1).mean()))*100

dfFinal["Sem_Vendas_120"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Sem_Vendas'].shift(1).transform(lambda x: x.rolling(window=120, min_periods=1).mean()))*100

dfFinal["Sem_Vendas_180"] = (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['Sem_Vendas'].shift(1).transform(lambda x: x.rolling(window=180, min_periods=1).mean()))*100


> - Vendas perdidas

In [52]:
dfFinal['ROTURA_fds'] = dfFinal[dfFinal['DATA'].dt.weekday.isin([4,5,6])]["ROTURA"].copy()
dfFinal['ROTURA_semana'] = dfFinal[dfFinal['DATA'].dt.weekday.isin([0,1,2,3])]["ROTURA"].copy()

In [53]:
'''Mediana de fins de semana e mediana de semana a multiplicar pelo nº de dias em que há rotura, soma dos valores para
ter as perdas de vendas estimadas'''

#mediana fds*roturas fds + mediana semana*roturas semana

dfFinal["Vendas_Perdidas_30"] = ((dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA_fds'].shift(1).transform(lambda x: x.rolling(window=30, min_periods=1).sum()) * dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds'].shift(1).transform(lambda x: x.rolling(window=30, min_periods=1).median())) + 
                                (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA_semana'].shift(1).transform(lambda x: x.rolling(window=30, min_periods=1).sum()) * dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana'].shift(1).transform(lambda x: x.rolling(window=30, min_periods=1).median()))
)
dfFinal["Vendas_Perdidas_60"] = ((dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA_fds'].shift(1).transform(lambda x: x.rolling(window=60, min_periods=1).sum()) * dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds'].shift(1).transform(lambda x: x.rolling(window=60, min_periods=1).median())) + 
                                (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA_semana'].shift(1).transform(lambda x: x.rolling(window=60, min_periods=1).sum()) * dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana'].shift(1).transform(lambda x: x.rolling(window=60, min_periods=1).median()))
)
dfFinal["Vendas_Perdidas_120"] = ((dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA_fds'].shift(1).transform(lambda x: x.rolling(window=120, min_periods=1).sum()) * dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds'].shift(1).transform(lambda x: x.rolling(window=120, min_periods=1).median())) + 
                                 (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA_semana'].shift(1).transform(lambda x: x.rolling(window=120, min_periods=1).sum()) * dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana'].shift(1).transform(lambda x: x.rolling(window=120, min_periods=1).median()))
)
dfFinal["Vendas_Perdidas_180"] = ((dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA_fds'].shift(1).transform(lambda x: x.rolling(window=180, min_periods=1).sum()) * dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_fds'].shift(1).transform(lambda x: x.rolling(window=180, min_periods=1).median())) + 
                                 (dfFinal.groupby(['DESC_ARTIGO', "STORE"])['ROTURA_semana'].shift(1).transform(lambda x: x.rolling(window=180, min_periods=1).sum()) * dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT_semana'].shift(1).transform(lambda x: x.rolling(window=180, min_periods=1).median()))
)

# Métricas até 10 dias antes:

- INSTRANSIT
- EXPECTED
- SELLOUT
- CICLOS
- Dias para Rotura
- Adequação

In [55]:
# Quantos dias antes:

diaI=1         #dia inicial
diaF=10        #dia final

> Função

In [56]:
# Função para colunas de dias anteriores
def dias(df, dia, coluna):         #dia é quantos dias antes
    a=int(dia)

    valores = df.groupby(['DESC_ARTIGO', 'STORE'])[coluna].transform(lambda x: x.shift(a))
    valores[:a] = np.nan
    
    df.loc[:,'%s_%s_Dias_Antes' % (coluna, a)] = valores

> - SELLOUTS

In [57]:
%%time
# Usar função para sellouts até 10 dias antes

for i in range(diaI+1, diaF+1):
    dias(dfFinal, i, "SELLOUT")

CPU times: total: 328 ms
Wall time: 348 ms


> - STOCKS

In [58]:
# Usar função para Stocks até 10 dias antes

for i in range(diaI, diaF+1):
    dias(dfFinal, i, "STOCK")

> > - Ordenar

> - INTRANSIT e EXPECTED

In [59]:
# Usar função para Trânsito até 10 dias antes


for i in range(diaI, diaF+1):
    dias(dfFinal, i, "INTRANSIT")
    
for i in range(diaI, diaF+1):
    dias(dfFinal, i, "EXPECTED")

> - STK

In [60]:
# STK do dia = soma dos stocks em loja com os stocks em trânsito no próprio dia

dfFinal["STK"] = dfFinal["STOCK"] + dfFinal["INTRANSIT"] + dfFinal["EXPECTED"]

for i in range(diaI, diaF+1):
    dias(dfFinal, i, "STK")

> - MSA

In [61]:
# MSA do dia = média dos sellouts dos 10 dias anteriores ao dia em causa

dfFinal["MSA10"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).mean())
dfFinal["MSA10Dp"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).std())

for i in range(diaI, diaF+1):
    dias(dfFinal, i, "MSA10")

    
dfFinal["MSA20"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=20).mean())
dfFinal["MSA20Dp"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=20).std())
  
for i in range(diaI, diaF+1):
    dias(dfFinal, i, "MSA20")

> - CICLOS

In [62]:
# Coluna de Ciclos de reposição

dfFinal["CICLOS"] = dfFinal["STOCK"]/dfFinal["PRES_STOCK"]

for i in range(diaI, diaF+1):
    dias(dfFinal, i, "CICLOS")

> - Dias para rotura de Stock

In [63]:
dfFinal = dfFinal.copy()
# Dias para a rotura mas com o Sellout médio (móvel) dos últimos 10 dias 
dfFinal["Dias_para_Rotura_Stock"] = dfFinal["STOCK"] / dfFinal["MSA10"]

for i in range(diaI, diaF+1):
    dias(dfFinal, i, "Dias_para_Rotura_Stock")

> - Dias para rotura de Linear

In [64]:
# Definir a métrica: Preslinear / med(Sellouts 10 dias)
dfFinal['Dias_Duração_Linear'] = dfFinal["PRES_STOCK"] / dfFinal["MSA10"]

for i in range(diaI, diaF+1):
    dias(dfFinal, i, "Dias_Duração_Linear")

> - Adequação de Stock

In [65]:
# Coluna de adequação de stock


dfFinal["Adequação"]= np.where(dfFinal["CICLOS"] > 1.1, "Stock Suficiente", 
                      np.where((dfFinal["CICLOS"] <= 1.1) & (dfFinal["INTRANSIT"]+dfFinal["EXPECTED"]+dfFinal["STOCK"]>=dfFinal["PRES_STOCK"]), "Stock Insuf c Forn Adequado", 
                      np.where((dfFinal["CICLOS"] <= 1.1) & (dfFinal["INTRANSIT"]+dfFinal["EXPECTED"]+dfFinal["STOCK"]<dfFinal["PRES_STOCK"]), "Stock Insuf c Forn Desadequado", 
                      "")))

for i in range(diaI, diaF+1):
    dias(dfFinal, i, "Adequação")

>- Balance: sellout / soma stock disponível mais transito.

In [66]:
# Colunas de balanço


# Balance do dia = razão entre o sellout médio e o stock para o dia actual
dfFinal["Balance"] =  dfFinal["MSA10"] / dfFinal["STK"]

for i in range(diaI, diaF+1):
    
    
    valores = (i+1) * dfFinal["Balance"].shift(i)
    valores[:i] = np.nan
    
    dfFinal.loc[:,'%s_%s_Dias_Antes' % ("Balance", i)] = valores
    
    


>- Balance optimizado

In [67]:


    
dfFinal["Balance_Optimized"] = np.where((dfFinal["MSA10Dp"] / dfFinal["MSA10"]) * 100 > 100, dfFinal["MSA20"] / dfFinal["STK"],
                                dfFinal["MSA10"] / dfFinal["STK"])   

for i in range(diaI, diaF+1):
    
    
    valores = (i+1) * dfFinal["Balance_Optimized"].shift(i)
    valores[:i] = np.nan
    
    dfFinal.loc[:,'%s_%s_Dias_Antes' % ("Balance_Optimized", i)] = valores

> - Mediano é com o mínimo dos ultimos 10 dias

In [68]:
# Coluna de adequação de stock


# MSA_med do dia = mediana dos sellouts dos 10 dias anteriores (exclui o próprio dia)
# dfFinal["MdSA"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).median())


# Balance do dia = razão entre o sellout médio e o stock para o dia actual
dfFinal["Balance_Mediano"] =  dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).median()) / dfFinal["STK"]

for i in range(diaI, diaF+1):
    
    
    valores = (i+1) * dfFinal["Balance_Mediano"].shift(i)
    valores[:i] = np.nan
    
    dfFinal.loc[:,'%s_%s_Dias_Antes' % ("Balance_Mediano", i)] = valores



> - Liberal é com o mínimo dos ultimos 10 dias

In [69]:
# Liberal


# MSA do dia = média dos stocks dos 10 dias anteriores (exclui o próprio dia)
#dfFinal["LSA"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).min())


# Balance do dia = razão entre o sellout médio e o stock para o dia actual
dfFinal["Balance_Liberal"] =  dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).min()) / dfFinal["STK"]

for i in range(diaI, diaF+1):
    
    
    valores = (i+1) * dfFinal["Balance_Liberal"].shift(i)
    valores[:i] = np.nan
    
    dfFinal.loc[:,'%s_%s_Dias_Antes' % ("Balance_Liberal", i)] = valores



> - Conservador é com o máximo dos ultimos 10 dias

In [70]:
# Conservador

# MSA_max do dia = máximo dos stocks dos 10 dias anteriores (exclui o próprio dia)
#dfFinal["CSA"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).max())
 
    
# Balance do dia = razão entre o sellout médio e o stock para o dia actual
dfFinal["Balance_Conservador"] =  dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).max()) / dfFinal["STK"]

for i in range(diaI, diaF+1):
    
    # Multiplicar os balances pelo número de dias+1 antes do dia actual
    valores = (i+1) * dfFinal["Balance_Conservador"].shift(i)
    valores[:i] = np.nan
    
    dfFinal.loc[:,'%s_%s_Dias_Antes' % ("Balance_Conservador", i)] = valores



# Escrever

- 0's e 1's

In [72]:
# Novo dataframe que apenas inclui dias em que existe a 1ª rotura e o dia anterior a essa rotura

# Tirar 1's depois do primeiro
dfFinalLimitado = dfFinal[~((dfFinal["ROTURA"] == 1) & (dfFinal["ROTURA"].shift(1) == 1))].copy()

# Apenas incluir primeira rotura
dfFinalLimitadoRoturas = dfFinal[((dfFinal["ROTURA"] == 1) & (dfFinal["ROTURA"].shift(1) == 0))].copy() #| ((dfFinal["ROTURA"] == 0) & (dfFinal["ROTURA"].shift(-1) == 1))]

- Dias certos

In [73]:
dfFinal2 = dfFinal.loc[(dfFinal['DATA'] >= '2023-01-01') ].copy()
#df_DiasCertos = dfFinalLimitado.loc[(dfFinalLimitado['DATA'] >= '2023-01-01') ].copy()
#df_RoturasDiasCertos = dfFinalLimitadoRoturas.loc[(dfFinalLimitadoRoturas['DATA'] >= '2023-01-01') ].copy()

In [76]:
dfFinal2.DATA.unique()

array(['2023-01-01T00:00:00.000000000', '2023-01-02T00:00:00.000000000',
       '2023-01-03T00:00:00.000000000', '2023-01-04T00:00:00.000000000',
       '2023-01-05T00:00:00.000000000', '2023-01-06T00:00:00.000000000',
       '2023-01-07T00:00:00.000000000', '2023-01-08T00:00:00.000000000',
       '2023-01-09T00:00:00.000000000', '2023-01-10T00:00:00.000000000',
       '2023-01-11T00:00:00.000000000', '2023-01-12T00:00:00.000000000',
       '2023-01-13T00:00:00.000000000', '2023-01-14T00:00:00.000000000',
       '2023-01-15T00:00:00.000000000', '2023-01-16T00:00:00.000000000',
       '2023-01-17T00:00:00.000000000', '2023-01-18T00:00:00.000000000',
       '2023-01-19T00:00:00.000000000', '2023-01-20T00:00:00.000000000',
       '2023-01-21T00:00:00.000000000', '2023-01-22T00:00:00.000000000',
       '2023-01-23T00:00:00.000000000', '2023-01-24T00:00:00.000000000',
       '2023-01-25T00:00:00.000000000', '2023-01-26T00:00:00.000000000',
       '2023-01-27T00:00:00.000000000', '2023-01-28

- Passar para csv

In [88]:
%%time

escrever_csv(filtered_oi, "Métricas3060120_Piloto")

CPU times: total: 219 ms
Wall time: 524 ms


In [55]:
escrever_csv(dfFinal2, "Mil_MétricasCerto07")
#escrever_csv(df_DiasCertos, "StocksDelta_2023_10prod_Limpo")
#escrever_csv(df_RoturasDiasCertos, "StocksDelta_2023_10prod_Roturas")

In [4]:
def escrever_excel(dataFrame, nomeFicheiro):
    dataFrame.to_excel('D:\\B&N Dados\\Delta\\Piloto\\%s.xlsx' %nomeFicheiro, index=False)
    

In [5]:
escrever_excel(oi, "ParaTestarCoisas")

In [4]:
oiEscrever = oi[("2023-04-14"<=oi["DATA"]<="2023-04-16") or ("2023-04-21"<=oi["DATA"]<="2023-04-23") or ("2023-04-28"<=oi["DATA"]<="2023-04-30") or ("2023-05-05"<=oi["DATA"]<="2023-05-07")]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [87]:
filtered_oi = oi[oi["DATA"].between("2023-04-14", "2023-04-16") |
                  oi["DATA"].between("2023-04-21", "2023-04-23") |
                  oi["DATA"].between("2023-04-28", "2023-04-30") |
                  oi["DATA"].between("2023-05-05", "2023-05-07")]